<a href="https://colab.research.google.com/github/Yashasvigm14/Sleep-Score-Prediction/blob/main/Copy_of_Sleep_Score_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_sleep_score(data):
    # Default values
    default_values = {
        "total_sleep_time": 420,
        "time_in_bed": 500,
        "rem_sleep": 80,
        "deep_sleep": 100,
        "light_sleep": 200,
        "awake_time": 50,
        "active_sleep": 30,
        "steps": 8000,
        "calories_burned": 2000,
        "exercise_minutes": 30,
        "heart_rate_avg": 75,
        "heart_rate_variability_avg": 60,
        "stress_level": 5,
    }

    # to handle missing values
    def get_value(key, data_dict, default):
        value = data_dict.get(key, default)
        return value if value is not None else default

    # Sleep duration scoring function
    def sleep_duration_score(total_sleep_time, lower_bound=420, upper_bound=540):
        if total_sleep_time < lower_bound:
            return total_sleep_time / lower_bound  # Penalize insufficient sleep
        elif total_sleep_time > upper_bound:
            return max(1 - (total_sleep_time - upper_bound) / upper_bound, 0)  # Penalize oversleeping
        else:
            return 1.0  # Optimal sleep range

    # Sleep efficiency penalty function
    def sleep_efficiency_penalty(total_sleep_time, time_in_bed, efficiency_threshold=85):
        if time_in_bed <= 0:
            return 1.0
        efficiency = (total_sleep_time / time_in_bed) * 100
        if efficiency < efficiency_threshold:
            return efficiency / efficiency_threshold  # Penalize for low efficiency
        return 1.0  # No penalty for good efficiency

    # Extract sleep data
    sleep_data = data.get('sleep_data', {})
    total_sleep_time = get_value('total_sleep_time', sleep_data, default_values["total_sleep_time"])
    time_in_bed = get_value('time_in_bed', sleep_data, default_values["time_in_bed"])
    sleep_efficiency = (total_sleep_time / time_in_bed) * 100 if time_in_bed > 0 else 0  # Dynamically calculated

    # Extract and calculate sleep stages
    sleep_stages = sleep_data.get('sleep_stages', {})
    rem_sleep = get_value('rem_sleep', sleep_stages, default_values["rem_sleep"])
    non_rem_sleep = sleep_stages.get('non_rem_sleep', {})
    light_sleep = get_value('light_sleep', non_rem_sleep, default_values["light_sleep"])
    deep_sleep = get_value('deep_sleep', non_rem_sleep, default_values["deep_sleep"])
    calculated_non_rem_sleep = light_sleep + deep_sleep  # Dynamically calculated

    # Extract other parameters
    awake_time = get_value('awake_time', sleep_stages, default_values["awake_time"])
    active_sleep = get_value('active_sleep', sleep_data, default_values["active_sleep"])
    steps = get_value('steps', data, default_values["steps"])
    calories_burned = get_value('calories_burned', data, default_values["calories_burned"])
    exercise_minutes = get_value('exercise_minutes', data, default_values["exercise_minutes"])
    heart_rate_avg = get_value('heart_rate_avg', data, default_values["heart_rate_avg"])
    heart_rate_variability_avg = get_value('heart_rate_variability_avg', data, default_values["heart_rate_variability_avg"])
    stress_level = get_value('stress_level', data, default_values["stress_level"])

    # Grouped parameters
    fitness = (steps / 10000 + calories_burned / 2500 + exercise_minutes / 30 + (1 - abs(heart_rate_avg - 60) / 40)) / 4
    restlessness = 1 - (awake_time / 60 + active_sleep / total_sleep_time) / 2
    stress_recovery = (1 - stress_level / 10 + heart_rate_variability_avg / 70) / 2

    # Parameter weights
    weights = {
        "total_sleep_time": 0.25,
        "sleep_efficiency": 0.15,
        "sleep_stages": 0.25,
        "restlessness": 0.10,
        "fitness": 0.10,
        "stress_recovery": 0.10,
    }

    # Calculate score
    total_sleep_score = sleep_duration_score(total_sleep_time) * sleep_efficiency_penalty(total_sleep_time, time_in_bed)
    score = 0
    score += weights["total_sleep_time"] * total_sleep_score * 100
    score += weights["sleep_efficiency"] * (sleep_efficiency / 100) * 100
    score += weights["sleep_stages"] * ((rem_sleep / 90 + deep_sleep / 120 + light_sleep / 210) / 3) * 100
    score += weights["restlessness"] * restlessness * 100
    score += weights["fitness"] * min(fitness, 1.0) * 100
    score += weights["stress_recovery"] * min(stress_recovery, 1.0) * 100

    # Ensure score is within bounds
    return round(max(0, min(score, 100)), 2)

# Example Input
input_data = {
    "steps": 10000,
    "calories_burned": 2500,
    "exercise_minutes": 40,
    "heart_rate_avg": 65,
    "heart_rate_variability_avg": 70,
    "stress_level": 2,
    "sleep_data": {
        "total_sleep_time": 450,
        "time_in_bed": 530,
        "sleep_stages": {
            "rem_sleep": None,  # Missing value
            "non_rem_sleep": {
                "light_sleep": 210,
                "deep_sleep": None
            },
            "awake_time": 50
        }
    }
}

# Calculate sleep score
print(f"Sleep Score: {calculate_sleep_score(input_data)}")


Sleep Score: 84.89
